In [1]:
import ROOT
import root_numpy as rnpy
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn

Welcome to ROOTaaS 6.06/08


In [2]:
d1 = pd.DataFrame(rnpy.root2array("/mnt/t3nfs01/data01/shome/jpata/btv/gc/TagVarExtractor/GCa08e5e237323/TT_TuneCUETP8M1_13TeV-powheg-pythia8/job_0_out.root",
                                 treename = "tagVars/ttree")
                 )

In [3]:
d2 = pd.DataFrame(rnpy.root2array("/mnt/t3nfs01/data01/shome/jpata/btv/gc/TagVarExtractor/GCa08e5e237323/TT_TuneCUETP8M1_13TeV-powheg-pythia8/job_0_out.root",
                                 treename = "tagVars/ttree_track"))

/swshare/anaconda/lib/python2.7/site-packages/root_numpy/_tree.py:209: RuntimeWarning: ignoring duplicate branch named 'Track_nHitPixel'
  warn_missing_tree)


In [5]:
jets_b = []
jets_l = []
jets_c = []

# iterate over the jet list
for irow, row in d1.iterrows():
    jetIndex = int(row.Jet_jetIndex)
    flavour = int(row["Jet_flavour"])
    
    # scan the track list to extract the tracks belonging to this jet
    sel = d2["Track_jetIndex"] == jetIndex
    
    if abs(flavour) == 5:
        jets = jets_b
    elif abs(flavour) == 4:
        jets = jets_c
    else:
        jets = jets_l
        
    #create jet tuple
    jets += [(
        (row["Jet_pt"], row["Jet_eta"], row["Jet_phi"], row["Jet_mass"]),
        int(row["Jet_flavour"]),
        d2[sel][["Track_pt", "Track_eta", "Track_phi", "Track_dxy", "Track_dz", "Track_IP", "Track_IP2D", "Track_length"]].as_matrix()
    )]
    if irow >= 50000:
        break

In [29]:
np.append(jets_b[1][2],[[0,0,0,1,2,3,4,5]], axis = 0)

array([[  3.90625000e+00,   9.05117929e-01,  -8.08425605e-01,
         -3.09375003e-02,  -8.10937509e-02,   6.43086731e-02,
          3.09375040e-02,   3.26143235e-01],
       [  1.36230469e+00,   6.89413130e-01,  -1.02835882e+00,
          1.60546880e-02,  -2.72216188e-04,   1.60561614e-02,
          1.60546862e-02,   2.08897203e-01],
       [  2.15039062e+00,   6.20014012e-01,  -1.13841820e+00,
          2.89373063e-02,   8.33480246e-03,   2.97615770e-02,
          2.89373696e-02,   1.71388343e-01],
       [  5.20703125e+00,   5.87786496e-01,  -1.14466190e+00,
          2.43558865e-02,   2.48010978e-02,   3.21964398e-02,
          2.43558735e-02,   1.67140812e-01],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.00000000e+00,   2.00000000e+00,   3.00000000e+00,
          4.00000000e+00,   5.00000000e+00]])

In [23]:
jets_b[1][2][1]

array([  1.36230469e+00,   6.89413130e-01,  -1.02835882e+00,
         1.60546880e-02,  -2.72216188e-04,   1.60561614e-02,
         1.60546862e-02,   2.08897203e-01], dtype=float32)

In [ ]:
# quicker way to do the partitioning: iterate though the jet tracks list,
# append each track and its parameters to the jet it belongs to (all jets in one big frame)
# at the end, divide the jets into their types, according to their jetIndex